Wandb installation

In [ ]:
!pip install wandb

     |████████████████████████████████| 2.1MB 3.8MB/s 
     |████████████████████████████████| 133kB 17.7MB/s 
     |████████████████████████████████| 163kB 16.3MB/s 
     |████████████████████████████████| 102kB 6.9MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=cf1d2cedcbc566f9d8b1869b795defa6bf9eae75c8dc494dc66934cb41a7cf50
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=348d177ada95a89a2452eb07bbd67ae61412c3d1082ebaa474e8ebe680eb6152
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


Import packages

In [ ]:
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adagrad
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image
from PIL import Image
import wandb
import math
from wandb.keras import WandbCallback

Nacitanie dat o obrazkoch

In [ ]:
df = pd.read_csv('./drive/MyDrive/Colab Notebooks/archive/nexet/nexet/train_boxes.csv')
df.head()

,image_filename,x0,y0,x1,y1,label,confidence
0,frame_817c47b8-22c4-438a-8dc6-0e3f67f299ee_000...,601.600000,270.355731,726.755556,421.185771,van,1.0
1,frame_817c47b8-22c4-438a-8dc6-0e3f67f299ee_000...,497.777778,308.774704,534.755556,338.656126,car,1.0
2,frame_817c47b8-22c4-438a-8dc6-0e3f67f299ee_000...,449.422222,310.197628,509.155556,358.577075,car,1.0
3,frame_a9110bf2-5252-4ec6-83c6-33b65d0fc04d_000...,711.111111,304.505929,786.488889,368.537549,car,1.0
4,frame_a9110bf2-5252-4ec6-83c6-33b65d0fc04d_000...,584.533333,307.351779,647.111111,358.577075,car,1.0


Nacitanie obrazkov a odstranenie malych obrazkov

In [ ]:
data = pd.DataFrame(columns=['image_filename', 'x0', 'y0', 'x1', 'y1', 'label', 'confidence'])
cropped_images = []
i8op
for index, row in df.iterrows():
  
  x = round(row['x0'])
  y = round(row['y0'])
  width = round(row['x1'] - row['x0'])
  height = round(row['y1'] - row['y0'])

  if width < 300:
        continue

  try:
    img = cv2.imread('./drive/MyDrive/Colab Notebooks/archive/nexet/nexet/nexet_2017_1/'+row['image_filename'])
    crop_img = img[y:y+height, x:x+width]
    data = data.append({'image_filename': row['image_filename'], 
                 'x0': row['x0'], 
                 'y0': row['y0'], 
                 'x1': row['x1'], 
                 'y1': row['y1'], 
                 'label': row['label'], 
                 'confidence': row['confidence'], }, ignore_index=True)
    cropped_images.append(crop_img)
  except: 
    pass

Normalizovanie obrazkov

In [ ]:
for i in range(len(cropped_images)):
  cropped_images[i] = np.array(cropped_images[i])/ 255.0

Transformovanie output labelov

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(data['label'])

Reshapovanie

In [ ]:
for i in range(len(cropped_images)):
  cropped_images[i] = np.resize( cropped_images[i], (200,200, 3))

Rozdelenie datasetu

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( np.array(cropped_images), np.array(transfomed_label), random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, random_state=42)

Definovanie modelu

In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    # compile model
    opt = Adagrad(lr=0.08325)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

Trenovanie modelu

In [ ]:
# define model
model = define_model()
# fit model
history = model.fit(X_train, y_train, epochs=80, batch_size=32, validation_data=(X_test, y_test), verbose=1)
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Wandb login

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Wand sweep config

In [ ]:
# define sweep
sweep_config = {
  "name": "NN-sweep-keras",
  "method": "random",
  "metric" : {
    'name': 'loss',
    'goal': 'minimize'   
    },
 "parameters" : {
    'optimizer': {
        'values': ['Adam', 'Adagrad', 'RMSprop', 'SGD']
        },

    'criterion': {
        'values': ['binary_crossentropy', 'categorical_crossentropy', 'mean_squared_error']
        },

    'h1': {
        'values': [8, 16, 32, 64,128,256]
        },

    'h2': {
        'values': [8, 16, 32, 64,128,256]
        },

    'h3': {
        'values': [8, 16, 32, 64,128,256]
        },

    'h4': {
        'values': [8, 16, 32, 64,128,256]
        },

    'learning_rate': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },

    'batch_size': {
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
      },

    'epochs': {
        'distribution': 'int_uniform',
        'min':80,
        'max': 100,
        }
    }   

}

Init sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config) 

Create sweep with ID: 9djxz19i
Sweep URL: https://wandb.ai/bombinko/uncategorized/sweeps/9djxz19i


Wandb train def

In [ ]:
def train():
    with wandb.init(config=sweep_config):
    # Specify the other hyperparameters to the configuration
        config = wandb.config
        # Add the config item (layers) to wandb
        if wandb.run:
            # Define the model
            model = Sequential()
            model.add(Conv2D(config.h1, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
            model.add(Conv2D(config.h1, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Dropout(0.2))
            model.add(Conv2D(config.h2, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
            model.add(Conv2D(config.h2, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Dropout(0.2))
            model.add(Conv2D(config.h3, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
            model.add(Conv2D(config.h3, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Dropout(0.2))
            model.add(Flatten())
            model.add(Dense(config.h4, activation='relu', kernel_initializer='he_uniform'))
            model.add(Dropout(0.2))
            model.add(Dense(5, activation='softmax'))
            model.compile(loss=config.criterion, optimizer=config.optimizer,metrics=['accuracy'])
            model.fit(X_train, y_train, epochs=config.epochs,validation_data=(X_test, y_test),callbacks=[WandbCallback()],verbose=0)

Run sweep

In [ ]:
wandb.agent(sweep_id, function=train, count=3)


wandb: Agent Starting Run: nkmmyl4o with config:
wandb: 	batch_size: 34
wandb: 	criterion: mean_squared_error
wandb: 	epochs: 89
wandb: 	h1: 8
wandb: 	h2: 128
wandb: 	h3: 64
wandb: 	h4: 16
wandb: 	learning_rate: 0.08662062667896303
wandb: 	optimizer: RMSprop


epoch,88
loss,0.01283
accuracy,0.96
val_loss,0.2606
val_accuracy,0.22222
_runtime,2123
_timestamp,1618482937
_step,88
best_val_loss,0.11997
best_epoch,16


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▅█▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▃▁▃▃▃▃▄▄▅▄▅▆▆▆▇▇▇█▇▇▇███▇▇▇▇▇▇▇▇████████
val_loss,█▃▂▁▁▁▁▁▁▁▂▁▁▁▂▂▁▄▃▂▂▂▂▄▂▂▆▄▃▄▃▃▃▃▃▃▃▃▄▅
val_accuracy,▁▁▅▇█▇▇▆▇▆▅▆▇▇▅▆▇▅▆▆▆▆▆▅▅▅▃▄▅▄▅▅▅▆▅▅▆▅▅▃
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: holzw940 with config:
wandb: 	batch_size: 73
wandb: 	criterion: categorical_crossentropy
wandb: 	epochs: 95
wandb: 	h1: 64
wandb: 	h2: 8
wandb: 	h3: 64
wandb: 	h4: 8
wandb: 	learning_rate: 0.09662537025648998
wandb: 	optimizer: Adagrad


epoch,94
loss,1.32062
accuracy,0.4
val_loss,1.41931
val_accuracy,0.5
_runtime,2528
_timestamp,1618485471
_step,94
best_val_loss,1.40095
best_epoch,88


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▄▄▃▃▄▃▃▃▃▃▃▃▂▃▃▂▄▂▃▂▂▃▃▂▃▃▂▂▃▂▂▂▂▂▂▁▁▁
accuracy,▄▅▄▂▃▂▂▃▂▃▃▃▂▁▂▅▁▁▂▂▃▃▃▂▂▂▂▂▂▃▅▅████████
val_loss,██▅▄▄▄▄▃▃▄▃▃▃▃▃▄▃▂▃▂▄▃▂▃▂▂▁▂▂▃▂▂▂▂▁▂▁▁▁▁
val_accuracy,▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▆▅▅▅▆▅▆▇████████████
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Agent Starting Run: fw2hq7r2 with config:
wandb: 	batch_size: 92
wandb: 	criterion: binary_crossentropy
wandb: 	epochs: 81
wandb: 	h1: 256
wandb: 	h2: 64
wandb: 	h3: 128
wandb: 	h4: 64
wandb: 	learning_rate: 0.002744675443289979
wandb: 	optimizer: Adam


wandb: Ctrl + C detected. Stopping sweep.
